In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import nltk
import re


In [2]:
#importing the dataset
dataset=pd.read_csv(r"D:\intern_project\amazonreviews.tsv",delimiter='\t',quoting=3)


In [3]:
#checking for null values
print(dataset.isnull().any())


asin            False
name             True
rating          False
date            False
verified        False
title            True
body             True
helpfulVotes     True
dtype: bool


In [4]:
#dropping unwanted columns
dataset=dataset.drop(columns =['name', 'date','asin','helpfulVotes'],axis=1) 


In [5]:
l1=dataset['rating'].tolist()
new_column=[]
for i in l1:
    if i>=3:
        new_column.append(1)
    else:
        new_column.append(0)


In [6]:
#converting above list into dataframe
df=pd.DataFrame(new_column,columns=['liked_or_not'])


In [7]:
#adding new column to dataset
dataset['liked_or_not']=df['liked_or_not']

print(dataset.isnull().sum())



rating           0
verified         0
title           14
body            21
liked_or_not     0
dtype: int64


In [8]:
#dropping columns with null values
dataset.dropna(inplace=True)


In [9]:
#removing unverified reviews
dataset['verified'].replace([False],[np.nan],inplace=True)

dataset.dropna(inplace=True)


In [10]:
#combining title abd body columns to form a new column "review"
dataset['Review'] = dataset[['title', 'body',]].agg(' '.join, axis=1)


In [11]:
#dropping unwanted columns
dataset.drop(columns=['title','body','verified','rating'],inplace=True)


In [12]:
#selecting review column for textprocessing
x=dataset.iloc[:,1].values


In [13]:
# text processing

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
new_dtst=[]
for i in range(len(x)):
    review=re.sub('[^a-zA-Z]','',str(x[i]))
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in set(stopwords.words('english'))]
    p=PorterStemmer()
    review=[p.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=''.join(review)
    new_dtst.append(review)
    


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#Creating "Bag of Words"
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
p=cv.fit_transform(new_dtst).toarray()
q=dataset.iloc[:,0:1].values


In [15]:
#splitting the data into testset and trainset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(p,q,test_size=0.2,random_state=0)


In [16]:
#model building

#importing libraries for building the model
import keras
from keras.models import Sequential
from keras.layers import Dense



Using TensorFlow backend.
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [17]:
#initialising the model
model=Sequential()


In [18]:
#adding input layer to the model
model.add(Dense(units=100,init='random_uniform',activation='relu'))
#adding hidden layer
model.add(Dense(units=30,init='random_uniform',activation='relu'))
#adding output layer
model.add(Dense(units=1,activation='sigmoid'))


C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100, activation="relu", kernel_initializer="random_uniform")`
  
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=30, activation="relu", kernel_initializer="random_uniform")`
  after removing the cwd from sys.path.


In [19]:
#configuring the learning process
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [20]:
#training the model
model.fit(x_train,y_train,epochs=50,batch_size=32)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50
48958/48958 [==============================] - 10s 198us/step - loss: 0.5347 - acc: 0.7664
Epoch 2/50
48958/48958 [==============================] - 8s 159us/step - loss: 0.5181 - acc: 0.7691
Epoch 3/50
48958/48958 [==============================] - 8s 164us/step - loss: 0.5137 - acc: 0.7760
Epoch 4/50
48958/48958 [==============================] - 8s 161us/step - loss: 0.5108 - acc: 0.7762
Epoch 5/50
48958/48958 [==============================] - 8s 157us/step - loss: 0.5095 - acc: 0.7762
Epoch 6/50
48958/48958 [==============================] - 8s 160us/step - loss: 0.5090 - acc: 0.7762
Epoch 7/50
48958/48958 [==============================] - 8s 172us/step - loss: 0.5082 - acc: 0.7762
Epoch 8/50
48958/48958 [==============================] - 8s 165us/step - loss: 0.5081 - acc: 0.7762
Epoch 9/50
48958/48958 [==============================] - 8s 161us/step - loss: 0.5082 - acc:

In [21]:
y_pred=model.predict(x_test)
y_pred=(y_pred>0.5)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [22]:
accuracy

0.7857843137254902

In [23]:
import pickle
pickle.dump(cv,open('count_vec.pkl','wb'))


In [24]:
#saving the model
model.save("my_model.h5")
